In [4]:
from collections import Counter

train = pd.read_csv('data/train.csv', dtype={'Upc': object})
test = pd.read_csv('data/test.csv', dtype={'Upc': object})
train_visits = train.VisitNumber.values
test_visits = test.VisitNumber.values
train = train.drop('TripType', axis=1)

data = pd.concat([train, test])

def check_digit(x):
    try:
        odd = map(int, ','.join(x[-1::-2]).split(','))
        even = map(int, ','.join(x[-2::-2]).split(','))
        sum_odd3 = sum(odd) * 3
        total = sum_odd3 + sum(even)
        rem = total % 10
        if rem == 0:
            return rem
        return 10 - rem
    except:
        return -9999

data['check'] = data.Upc.apply(check_digit)

In [5]:
def full_upc(x):
    try:
        if len(x) < 12:
            missing = 11 - len(x)
            zeros = ['0'] * missing
            xx = zeros + ','.join(x).split(',') + [str(check_digit(x))]
            xx = ''.join(xx)
            return xx
    except:
        return -9999

data['full_upc'] = data.Upc.apply(full_upc)

data = data.drop('Upc', axis=1)

In [6]:
def company(x):
    try:
        p = x[:6]
        if p == '000000':
            return x[-5]
        return p
    except:
        return -9999

data['company'] = data.full_upc.apply(company)

data.to_csv('data/fulldata.csv', index=False)

In [7]:
data

,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,check,full_upc,company
0,5,Friday,-1,FINANCIAL SERVICES,1000.0,7,681131529297,681131
1,7,Friday,1,SHOES,8931.0,9,605388159809,605388
2,7,Friday,1,PERSONAL CARE,4504.0,2,074108110992,074108
3,8,Friday,2,PAINT AND ACCESSORIES,3565.0,2,022384035102,022384
4,8,Friday,2,PAINT AND ACCESSORIES,1017.0,1,020066137441,020066
5,8,Friday,2,PAINT AND ACCESSORIES,1017.0,5,020066187835,020066
6,8,Friday,1,PAINT AND ACCESSORIES,1017.0,4,020066137434,020066
7,8,Friday,1,PAINT AND ACCESSORIES,2802.0,6,070048027376,070048
8,8,Friday,1,PAINT AND ACCESSORIES,4501.0,5,022384953185,022384
9,8,Friday,-1,PAINT AND ACCESSORIES,3565.0,5,022384002005,022384


In [8]:
from sklearn import preprocessing
data = data.fillna(-9999)
for col in data.columns:
    print(col)
    if col not in [u'VisitNumber', u'ScanCount']:
        lbl = preprocessing.LabelEncoder()
        data[col] = lbl.fit_transform(data[col].values)


data.to_csv('data/full_data_enc.csv', index=False)

VisitNumber
Weekday
ScanCount
DepartmentDescription


TypeError: '<' not supported between instances of 'int' and 'str'